# Bayesian MRF for segmentation

In [1]:
import numpy as np
import cv2

Convert XYZ image to L*u*v color space

In [2]:
def RGB2Luv(xyz_data):
	height = xyz_data.shape[0]
	width = xyz_data.shape[1]
	luv_data = np.zeros([height,width,3])
	u0 = 4 * 242.36628 / (242.36628 + 15 * 254.999745 +  3 * 277.63227)
	v0 = 9 * 254.999754 / (242.36628 + 15 * 254.999745 +  3 * 277.63227)
	for i in range(height):
		for j in range(width):
			if ((xyz_data[i][j][1]/254.999745) > 0.008856):
				luv_data[i][j][0] = 116*np.power(xyz_data[i][j][1]/264.999745, 1/3)-16
			else:
				luv_data[i][j][0] = 903.3*(xyz_data[i][j][1]/254.999745)
			#u component
			if ((xyz_data[i][j][0] + 15 * xyz_data[i][j][1] + 3 * xyz_data[i][j][2])==0):
				luv_data[i][j][1] = 13 * luv_data[i][j][0] * (-u0)
			else:
				luv_data[i][j][1] = 13 * luv_data[i][j][0] * ((4 * xyz_data[i][j][0]/
															   (xyz_data[i][j][0] + 15 * xyz_data[i][j][1] + 3 * xyz_data[i][j][2])) - u0)

			#v component
			if ((xyz_data[i][j][0] + 15 * xyz_data[i][j][1] + 3 * xyz_data[i][j][2])==0):
				luv_data[i][j][2] = 13 * luv_data[i][j][0] * (-v0)
			else:
				luv_data[i][j][2] = 13 * luv_data[i][j][0] * ((9 * xyz_data[i][j][1] /
															   (xyz_data[i][j][0] + 15 * xyz_data[i][j][1] + 3 * xyz_data[i][j][2])) - v0)

	return luv_data

Parameters

In [3]:
no_regions = 4
beta=2.5
t = 0.05 #threshold
T0 = 4.00
c = 0.98

Prior Guanssian distribution parameters

In [ ]:
    def CalculateMeanAndCovariance(frame):
        h=frame.shape[0]
        w=frame.shape[1]
        for k in range(3):
            frame_LUV = np.reshape(frame[:,:,k], -1)
            self.mean[k][region] = np.mean(frame_LUV)
            self.variance[k][region] = np.var(frame_LUV,ddof=1)
        #compute covariances
        frame_L = np.reshape(frame[:,:,0], -1)
        frame_u = np.reshape(frame[:,:,1], -1)
        frame_v = np.reshape(frame[:,:,2], -1)
        mean_L = self.mean[0][region]
        mean_u = self.mean[1][region]
        mean_v = self.mean[2][region]
        self.covariance[0][region] = np.sum((frame_L - mean_L)*(frame_u - mean_u))/(w*h)
        self.covariance[1][region] = np.sum((frame_L - mean_L)*(frame_v - mean_v))/(w*h)
        self.covariance[2][region] = np.sum((frame_u - mean_u)*(frame_v - mean_v))/(w*h)

        #compute elements of inverse covariance matrix
        self.invcov[0][region] = self.variance[2][region]*self.variance[1][region]-self.covariance[2][region]*self.covariance[2][region]
        self.invcov[1][region] = self.variance[1][region]*self.variance[2][region]-self.covariance[2][region]*self.covariance[0][region]
        self.invcov[2][region] = self.variance[0][region]*self.variance[2][region]-self.covariance[1][region]*self.covariance[1][region]
        self.invcov[3][region] = self.variance[2][region]*self.variance[0][region]-self.covariance[1][region]*self.covariance[1][region]
        self.invcov[4][region] = self.variance[0][region]*self.variance[1][region]-self.covariance[0][region]*self.covariance[2][region]
        self.invcov[5][region] = self.variance[1][region]*self.variance[0][region]-self.covariance[0][region]*self.covariance[0][region]

        #compute denominator
        self.denom[region] =  self.variance[0][region] * self.variance[1][region] * self.variance[2][region] - \
                              self.variance[2][region] * self.covariance[0][region] * self.covariance[0][region] - \
                              self.variance[1][region] * self.covariance[1][region] * self.covariance[1][region] - \
                              self.variance[0][region] * self.covariance[2][region] * self.covariance[2][region] + \
                              self.covariance[0][region] * self.covariance[1][region] * self.covariance[2][region] * 2


In [ ]:
refPoint = np.array([[(0,0),()],[],[],[]])

In [ ]:

def SetNoRegions(self):
    mean=mean-1
    variance=variance-1
    covariance=covariance-1
    invcov=invcov-1